# Retrieval Augmented Generation for Question Answering on Documents

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

## Importing `llama3.1:8b` Model

In [2]:
model=ChatOllama(model='llama3.1:8b')

## Loading the Document. `Leave Policy Bajra Technologies`

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "/home/bikasherl/Desktop/Week 10/LEAVE POLICY BAJRA TECHNOLOGIES 2081_82.docx.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load()


# print(pages[0].page_content)

## Chunking, Embedding and Storing

In [4]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))
retriever = vectorstore.as_retriever()



# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use one sentence maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )


# question_answer_chain = create_stuff_documents_chain(model, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)


# response = rag_chain.invoke({"input": "What is this document about? Please answer in one sentence."})
# response["answer"]

/home/bikasherl/miniconda3/envs/bajra/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Hybrid Search: Combining BM25 and Semantic Search for Better Results

In [5]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k =  2  # Retrieve top 2 results



# Initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                                       weights=[0.4, 0.6])

## Create History Aware Chatbot

In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# history_aware_retriever = create_history_aware_retriever(
#     model, retriever, contextualize_q_prompt
# )


# for hybrid search
history_aware_retriever = create_history_aware_retriever(
    model, ensemble_retriever, contextualize_q_prompt
)


In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [7]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "what are the different types of paid leaves available?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "Answer in list format."
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

There are several types of paid leaves available:

* Annual leave
* Sick leave
* Marriage/Honeymoon
* Accident cases
* Death of a family member
* Maternity


In [10]:
print(ai_msg_1['answer'])

There is a Floating leave which includes: a. Dashain - 5 working days, b. Tihar - 2 working days, c. Holi Purnima - 1 day, d. Nepali New Year - 1 day, and e. Floating Festive leaves - 4 days (which can be any festival/public holidays celebrated).


## Better Way to Make History Aware Chatbot

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)



In [9]:
user_inp=input("Ask me anything!")


conversational_rag_chain.invoke(
    {"input": user_inp},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'According to the context, here are some key points about the Leave Policy of Bajra Technologies:\n\n1. The calendar year for leave starts from July 16 to July 15 each year.\n2. Each employee must apply for leaves via the BizOS system and not via email.\n3. The maximum leave an employee can earn in a year is 26 working days (or 208 working hours).\n4. Annual leave:\n\t* Up to 3 days requires 5 working days prior notice.\n\t* 4 or more days in a row requires at least 4 weeks (or 20 working days) advance notice.\n5. Sick leave can be applied for at any time but must be approved in advance.\n6. Failure to get approval will result in unpaid leave.\n7. The company reserves the right to change the leave policy.\n8. Annual leaves should not be taken from October to December and during the last quarter of the fiscal year (Baisakh to Asar end).\n9. Leave should not be taken immediately before and after Dashain and Tihar unless unavoidable circumstances arise.\n10. During emergency situations, e

In [22]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: If i do overtime of 2 hours daily for 6 months continuously, how many days of paid leave do i get at the end?

AI: I don't know. The context doesn't provide information about how to calculate paid leave based on working extra hours.

User: 1 day has 8 working hours. any additional hours count as overtime. now calculate based on it.

AI: Since you've worked an average of 2 hours overtime per day, that's a total of 6*2 = 12 hours of overtime over 180 days (6 months).

Assuming a standard workweek has 40 working hours and 1 day has 8 working hours, 208 working hours in a year can be calculated as: 208/5 = 41.6 weeks.

Given that 1 day has 8 working hours, the maximum overtime hours in a year would be 8 * (52-41.6) = 16.48 hours, which is equivalent to approximately 2 hours of overtime per week.

Since you've worked an average of 2 hours of overtime daily, it's reasonable to assume that your total overtime hours are equivalent to some number of days' worth of leave. 

Let's calculate

## Function Call

In [40]:
from typing import Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions

model = OllamaFunctions(
    model="llama3.1:8b", 
    format="json"
    )

class ResponseFormatter(BaseModel):
    """Use this to answer about science"""

    question: str = Field(description = "Original question")
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    character: str = Field(description = "character of the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")

class Add(BaseModel):
    """Use this whenever addition problems occurs"""

    a: float = Field(description="First Number")
    b: float = Field(description="Second Number")
    outp: str = Field(description="add 5 and 10")


model_with_tools = model.bind_tools([ResponseFormatter,Joke,Add])

ai_msg = model_with_tools.invoke("What is the addition of 5 and 10?")

ai_msg.tool_calls[0]['args']


{'a': 5, 'b': 10, 'outp': ''}

## Langsmith Implementation

In [10]:
api_key='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'

In [13]:


# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY="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"
# LANGCHAIN_PROJECT="chatbotBajra"

In [9]:
from langsmith import traceable

@traceable # Auto-trace this function
def run_this(user_inp:str):
    return conversational_rag_chain.invoke(
        {"input": user_inp},
        config={
            "configurable": {"session_id": "abc123"}
        },  # constructs a key "abc123" in `store`.
    )["answer"]

user_inp=input("Ask me anything!")
run_this(user_inp)

'You are an employee of the company, as indicated by various provisions in the leave policy document.'